In [4]:
import os, sys

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [14]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# ---------------------
#  Generator
# ---------------------
    
    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)
    
# ---------------------
#  Discriminator
# ---------------------
    
    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):
        
        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

# ---------------------
#  Train Discriminator
# ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


# ---------------------
#  Train Generator
# ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

# ---------------------
#  Save and Plot images
# ---------------------
    save_dir = './star_date'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_64 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_46 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_65 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_47 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

99 [D loss: 0.018720, acc.: 100.00%] [G loss: 4.348802]
100 [D loss: 0.047903, acc.: 100.00%] [G loss: 4.921594]
101 [D loss: 0.025399, acc.: 100.00%] [G loss: 4.651134]
102 [D loss: 0.018870, acc.: 100.00%] [G loss: 4.805140]
103 [D loss: 0.011278, acc.: 100.00%] [G loss: 4.483665]
104 [D loss: 0.012082, acc.: 100.00%] [G loss: 4.378459]
105 [D loss: 0.009574, acc.: 100.00%] [G loss: 4.459136]
106 [D loss: 0.038109, acc.: 100.00%] [G loss: 4.919999]
107 [D loss: 0.013727, acc.: 100.00%] [G loss: 4.623889]
108 [D loss: 0.008067, acc.: 100.00%] [G loss: 4.586522]
109 [D loss: 0.027312, acc.: 100.00%] [G loss: 4.592647]
110 [D loss: 0.030663, acc.: 100.00%] [G loss: 5.247623]
111 [D loss: 0.060585, acc.: 100.00%] [G loss: 3.858781]
112 [D loss: 0.142449, acc.: 90.62%] [G loss: 4.980050]
113 [D loss: 0.022596, acc.: 100.00%] [G loss: 5.766153]
114 [D loss: 0.051390, acc.: 100.00%] [G loss: 4.380049]
115 [D loss: 0.070777, acc.: 96.88%] [G loss: 4.656982]
116 [D loss: 0.042968, acc.: 96.88

245 [D loss: 0.339780, acc.: 90.62%] [G loss: 2.509454]
246 [D loss: 0.787772, acc.: 53.12%] [G loss: 1.066015]
247 [D loss: 0.518340, acc.: 62.50%] [G loss: 2.649919]
248 [D loss: 0.812674, acc.: 65.62%] [G loss: 1.757363]
249 [D loss: 0.771805, acc.: 50.00%] [G loss: 1.995743]
250 [D loss: 0.519295, acc.: 75.00%] [G loss: 1.921980]
251 [D loss: 0.603881, acc.: 56.25%] [G loss: 1.419798]
252 [D loss: 0.416261, acc.: 78.12%] [G loss: 2.410903]
253 [D loss: 1.063776, acc.: 43.75%] [G loss: 1.321109]
254 [D loss: 0.628097, acc.: 59.38%] [G loss: 2.187459]
255 [D loss: 0.907933, acc.: 37.50%] [G loss: 1.318448]
256 [D loss: 0.813766, acc.: 46.88%] [G loss: 1.316566]
257 [D loss: 0.592540, acc.: 65.62%] [G loss: 1.585652]
258 [D loss: 0.882193, acc.: 37.50%] [G loss: 1.119047]
259 [D loss: 0.601517, acc.: 62.50%] [G loss: 1.558893]
260 [D loss: 1.018462, acc.: 28.12%] [G loss: 1.004415]
261 [D loss: 0.624121, acc.: 50.00%] [G loss: 1.876749]
262 [D loss: 0.686226, acc.: 56.25%] [G loss: 1.

398 [D loss: 0.671486, acc.: 43.75%] [G loss: 0.706260]
399 [D loss: 0.652347, acc.: 56.25%] [G loss: 0.699872]
400 [D loss: 0.692322, acc.: 40.62%] [G loss: 0.706250]
401 [D loss: 0.653144, acc.: 56.25%] [G loss: 0.697729]
402 [D loss: 0.650029, acc.: 43.75%] [G loss: 0.692295]
403 [D loss: 0.681457, acc.: 43.75%] [G loss: 0.673260]
404 [D loss: 0.701440, acc.: 50.00%] [G loss: 0.681028]
405 [D loss: 0.675240, acc.: 53.12%] [G loss: 0.705886]
406 [D loss: 0.642899, acc.: 50.00%] [G loss: 0.707080]
407 [D loss: 0.689372, acc.: 34.38%] [G loss: 0.707808]
408 [D loss: 0.672577, acc.: 40.62%] [G loss: 0.698239]
409 [D loss: 0.658038, acc.: 46.88%] [G loss: 0.684092]
410 [D loss: 0.668922, acc.: 43.75%] [G loss: 0.683078]
411 [D loss: 0.694774, acc.: 43.75%] [G loss: 0.688106]
412 [D loss: 0.676069, acc.: 46.88%] [G loss: 0.683372]
413 [D loss: 0.685049, acc.: 43.75%] [G loss: 0.691687]
414 [D loss: 0.672097, acc.: 43.75%] [G loss: 0.703709]
415 [D loss: 0.629906, acc.: 62.50%] [G loss: 0.

545 [D loss: 0.635931, acc.: 59.38%] [G loss: 0.740177]
546 [D loss: 0.670892, acc.: 59.38%] [G loss: 0.752345]
547 [D loss: 0.646640, acc.: 62.50%] [G loss: 0.760664]
548 [D loss: 0.681391, acc.: 50.00%] [G loss: 0.755092]
549 [D loss: 0.641068, acc.: 62.50%] [G loss: 0.708917]
550 [D loss: 0.660889, acc.: 53.12%] [G loss: 0.726078]
551 [D loss: 0.630716, acc.: 56.25%] [G loss: 0.761602]
552 [D loss: 0.648658, acc.: 62.50%] [G loss: 0.789065]
553 [D loss: 0.652120, acc.: 62.50%] [G loss: 0.731005]
554 [D loss: 0.635095, acc.: 56.25%] [G loss: 0.694864]
555 [D loss: 0.643889, acc.: 62.50%] [G loss: 0.730110]
556 [D loss: 0.641019, acc.: 53.12%] [G loss: 0.734291]
557 [D loss: 0.644672, acc.: 62.50%] [G loss: 0.743586]
558 [D loss: 0.650570, acc.: 59.38%] [G loss: 0.731987]
559 [D loss: 0.635137, acc.: 56.25%] [G loss: 0.735825]
560 [D loss: 0.678029, acc.: 50.00%] [G loss: 0.748818]
561 [D loss: 0.645442, acc.: 56.25%] [G loss: 0.741642]
562 [D loss: 0.644723, acc.: 62.50%] [G loss: 0.

...

29335 [D loss: 0.721188, acc.: 46.88%] [G loss: 0.875049]
29336 [D loss: 0.637441, acc.: 68.75%] [G loss: 0.805614]
29337 [D loss: 0.719744, acc.: 53.12%] [G loss: 0.841650]
29338 [D loss: 0.688893, acc.: 56.25%] [G loss: 0.821399]
29339 [D loss: 0.658071, acc.: 68.75%] [G loss: 0.876370]
29340 [D loss: 0.689388, acc.: 59.38%] [G loss: 0.850140]
29341 [D loss: 0.611359, acc.: 62.50%] [G loss: 0.868767]
29342 [D loss: 0.727782, acc.: 59.38%] [G loss: 0.903967]
29343 [D loss: 0.725356, acc.: 50.00%] [G loss: 0.900572]
29344 [D loss: 0.632336, acc.: 62.50%] [G loss: 0.837627]
29345 [D loss: 0.728314, acc.: 50.00%] [G loss: 0.811962]
29346 [D loss: 0.715258, acc.: 50.00%] [G loss: 0.975646]
29347 [D loss: 0.649319, acc.: 65.62%] [G loss: 0.919382]
29348 [D loss: 0.614991, acc.: 71.88%] [G loss: 0.929550]
29349 [D loss: 0.700044, acc.: 56.25%] [G loss: 0.936077]
29350 [D loss: 0.647590, acc.: 62.50%] [G loss: 0.835344]
29351 [D loss: 0.662491, acc.: 65.62%] [G loss: 0.824727]
29352 [D loss:

29481 [D loss: 0.645209, acc.: 68.75%] [G loss: 0.917988]
29482 [D loss: 0.573479, acc.: 68.75%] [G loss: 0.912496]
29483 [D loss: 0.750430, acc.: 40.62%] [G loss: 0.814948]
29484 [D loss: 0.686839, acc.: 56.25%] [G loss: 0.703161]
29485 [D loss: 0.628782, acc.: 65.62%] [G loss: 0.851489]
29486 [D loss: 0.665776, acc.: 59.38%] [G loss: 0.862322]
29487 [D loss: 0.683749, acc.: 56.25%] [G loss: 0.829436]
29488 [D loss: 0.791815, acc.: 40.62%] [G loss: 0.869169]
29489 [D loss: 0.612030, acc.: 75.00%] [G loss: 0.835205]
29490 [D loss: 0.634194, acc.: 68.75%] [G loss: 0.873018]
29491 [D loss: 0.721810, acc.: 40.62%] [G loss: 0.899595]
29492 [D loss: 0.840506, acc.: 53.12%] [G loss: 0.862961]
29493 [D loss: 0.639301, acc.: 71.88%] [G loss: 1.012821]
29494 [D loss: 0.707149, acc.: 46.88%] [G loss: 0.890222]
29495 [D loss: 0.719107, acc.: 50.00%] [G loss: 0.897185]
29496 [D loss: 0.618217, acc.: 71.88%] [G loss: 0.851509]
29497 [D loss: 0.605588, acc.: 75.00%] [G loss: 0.879326]
29498 [D loss:

29624 [D loss: 0.681753, acc.: 59.38%] [G loss: 0.851661]
29625 [D loss: 0.655765, acc.: 56.25%] [G loss: 0.884308]
29626 [D loss: 0.626408, acc.: 68.75%] [G loss: 0.858116]
29627 [D loss: 0.706500, acc.: 53.12%] [G loss: 0.874832]
29628 [D loss: 0.731176, acc.: 43.75%] [G loss: 0.922036]
29629 [D loss: 0.657945, acc.: 65.62%] [G loss: 0.885902]
29630 [D loss: 0.630380, acc.: 71.88%] [G loss: 0.883365]
29631 [D loss: 0.691515, acc.: 53.12%] [G loss: 0.826582]
29632 [D loss: 0.617022, acc.: 71.88%] [G loss: 0.866336]
29633 [D loss: 0.678632, acc.: 53.12%] [G loss: 0.824140]
29634 [D loss: 0.697376, acc.: 50.00%] [G loss: 0.938836]
29635 [D loss: 0.640292, acc.: 65.62%] [G loss: 0.856368]
29636 [D loss: 0.736458, acc.: 53.12%] [G loss: 0.841399]
29637 [D loss: 0.606532, acc.: 71.88%] [G loss: 0.873777]
29638 [D loss: 0.610911, acc.: 71.88%] [G loss: 0.861058]
29639 [D loss: 0.740421, acc.: 46.88%] [G loss: 0.848285]
29640 [D loss: 0.602813, acc.: 68.75%] [G loss: 0.908866]
29641 [D loss:

29767 [D loss: 0.721811, acc.: 43.75%] [G loss: 0.834444]
29768 [D loss: 0.679270, acc.: 53.12%] [G loss: 0.792013]
29769 [D loss: 0.637898, acc.: 65.62%] [G loss: 0.863679]
29770 [D loss: 0.692607, acc.: 65.62%] [G loss: 0.905840]
29771 [D loss: 0.636035, acc.: 65.62%] [G loss: 0.906906]
29772 [D loss: 0.687548, acc.: 53.12%] [G loss: 0.921809]
29773 [D loss: 0.616590, acc.: 68.75%] [G loss: 0.876514]
29774 [D loss: 0.630771, acc.: 68.75%] [G loss: 0.854651]
29775 [D loss: 0.758558, acc.: 46.88%] [G loss: 0.803709]
29776 [D loss: 0.725513, acc.: 50.00%] [G loss: 0.829992]
29777 [D loss: 0.669647, acc.: 65.62%] [G loss: 0.775853]
29778 [D loss: 0.690788, acc.: 65.62%] [G loss: 0.842119]
29779 [D loss: 0.596686, acc.: 68.75%] [G loss: 0.760782]
29780 [D loss: 0.678707, acc.: 62.50%] [G loss: 0.811703]
29781 [D loss: 0.675911, acc.: 59.38%] [G loss: 0.840006]
29782 [D loss: 0.693652, acc.: 59.38%] [G loss: 0.757537]
29783 [D loss: 0.661787, acc.: 62.50%] [G loss: 0.831403]
29784 [D loss:

29911 [D loss: 0.693819, acc.: 59.38%] [G loss: 0.853331]
29912 [D loss: 0.610152, acc.: 75.00%] [G loss: 0.834739]
29913 [D loss: 0.750318, acc.: 53.12%] [G loss: 0.919709]
29914 [D loss: 0.674708, acc.: 62.50%] [G loss: 0.864437]
29915 [D loss: 0.645398, acc.: 65.62%] [G loss: 0.825650]
29916 [D loss: 0.666878, acc.: 65.62%] [G loss: 0.798891]
29917 [D loss: 0.699755, acc.: 53.12%] [G loss: 0.766494]
29918 [D loss: 0.676779, acc.: 59.38%] [G loss: 0.836796]
29919 [D loss: 0.687516, acc.: 56.25%] [G loss: 0.815388]
29920 [D loss: 0.690002, acc.: 65.62%] [G loss: 0.929911]
29921 [D loss: 0.637594, acc.: 71.88%] [G loss: 0.881160]
29922 [D loss: 0.609263, acc.: 68.75%] [G loss: 0.893167]
29923 [D loss: 0.693449, acc.: 56.25%] [G loss: 0.747151]
29924 [D loss: 0.730212, acc.: 53.12%] [G loss: 0.876849]
29925 [D loss: 0.663734, acc.: 56.25%] [G loss: 0.796450]
29926 [D loss: 0.698164, acc.: 59.38%] [G loss: 0.843708]
29927 [D loss: 0.718839, acc.: 59.38%] [G loss: 0.902008]
29928 [D loss:

In [21]:
    def save_imgs(self, epoch):
        row, col = 5, 5
        noise = np.random.normal(0, 1, (row * col, 100))
        gen_imgs = self.generator.predict(noise)

        fig, axs = plt.subplots(row, col, figsize=(16, 6))
        cnt = 0
        for i in range(row):
            for j in range(col):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='ocean')
                axs[i,j].axis('off')
                cnt += 1
        print(fig.savefig("Star_date%d.png" % epoch))
        plt.close()